In [2]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.layers import Input, Dense
from keras.models import Model
from keras.callbacks import EarlyStopping
import numpy as np

# Load the data from the CSV file
data = pd.read_csv('Data.csv')

# Preprocess the data
scaler = MinMaxScaler(feature_range=(0,6))
preprocessed_data = scaler.fit_transform(data)

raw_data = preprocessed_data.tolist()
sequence_length = 10  # length of input sequence
data = []
for i in range(len(raw_data) - sequence_length):
    data.append(raw_data[i:i+sequence_length+1])
data = np.array(data)
X = data[:, :-1].reshape(-1, sequence_length, 1)
y = data[:, -1].reshape(-1, 1)

# Define the input shape
input_shape = (10,)

# Define the size of the encoded representation
encoding_dim = 256

input_layer = Input(shape=input_shape)

# Define the encoding layers
encoder_layer_1 = Dense(512, activation='relu')(input_layer)
encoder_layer_2 = Dense(encoding_dim, activation='relu')(encoder_layer_1)

# Define the decoding layers
decoder_layer_1 = Dense(512, activation='relu')(encoder_layer_2)
decoder_layer_2 = Dense(10)(decoder_layer_1)

# Define the autoencoder model
autoencoder_model = Model(input_layer, decoder_layer_2)

# Compile the model with a mean squared error loss function
autoencoder_model.compile(optimizer='adam', loss='binary_crossentropy')

# Print the model summary
autoencoder_model.summary()

# Split the data into training and validation sets
# Train the autoencoder model
autoencoder_model.fit(X, y,
                      epochs=100,
                      batch_size=32,
                      shuffle=True,
                      )

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 10)]              0         
                                                                 
 dense (Dense)               (None, 512)               5632      
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 dense_2 (Dense)             (None, 512)               131584    
                                                                 
 dense_3 (Dense)             (None, 10)                5130      
                                                                 
Total params: 273,674
Trainable params: 273,674
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
25/25 [==============================] - 1s

In [4]:
# Generate new data to be predicted
new_data = pd.DataFrame({'42.79': [1.56, 51.5, 1.39, 1.38, 1.06, 2.05, 3.91, 1.63, 1.39, 2.29]})

# Preprocess the new data using the same scaler object
preprocessed_new_data = scaler.transform(new_data)
test_data = preprocessed_new_data.tolist()
X_test = np.array(test_data).reshape(1, 10, 1)
# Predict the reconstructed output for the new data
reconstructed_output = autoencoder_model.predict([X_test])
# Inverse transform the reconstructed output using the scaler object
decoded_new_data = scaler.inverse_transform(reconstructed_output)

# Print the decoded new data
print(decoded_new_data)

1/1 [==============================] - 0s 18ms/step
[[3.440516  5.9705057 1.8134329 4.086042  2.2122383 3.2399466 4.0790887
  5.6320167 6.3999095 3.3116431]]
